In [111]:
import re
import string
from sklearn.model_selection import train_test_split
from nltk.util import ngrams
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.lm import Vocabulary, MLE
# import pandas as pd
from numpy.random import multinomial as multi
from random import randrange

# from keras.layers import LSTM

## Read, Parse and Divide Trump speech data

In [71]:
file=open("trumpspeeches.txt",'r')
text=file.read()

In [72]:
list_sent=list(sent_tokenize(text))

In [97]:
for sent in list_sent:
    sent=re.sub(r'[^\w]'," ",sent)
    list_sent.pop(0)
    list_sent.append(sent)

In [98]:
list_sent[0]

'  thank you so much'

In [99]:
text_train, text_test=train_test_split(list_sent,test_size=0.2,train_size=0.8, random_state=42)
print(len(text_train))
print(len(text_test))

13194
3299


In [100]:
text_train

['or stupid',
 'not going to happen anymore',
 'believe me',
 'he signed that',
 'our infrastructure is a disaster',
 'but we have so many people like that',
 'i mean you re in the crazy world of politics',
 'it s true',
 'i wonder if time magazine had that one to do over again what they d say',
 'we have leadership again no leadership respects us',
 'we mourn as one people for our nation s loss and pledge our support to any and all who need it',
 'i love all these people',
 'we gave them iraq that they ve been after for years',
 'we gave iran iraq',
 'and i was so nice',
 'they couldn t care less',
 'the worst',
 'you talk about a great franchise',
 'and you have a truly great quarterback',
 'very important',
 'right',
 'yeah go ahead finish it',
 'let me tell you something',
 'the judge has been very unfair',
 'politicians are all talk no action',
 'now hes saying well never pay for the wall',
 'nobody even knows why',
 'i can settle that case',
 'these are the stats',
 'we can t hav

## Q1

In [101]:
def gettoken(sent_tokens,if_sent=False):
    tokens=[]
    if(if_sent==False):
        for i in range(len(sent_tokens)):
            temp=word_tokenize(sent_tokens[i])
            tokens.append('<s>')
            tokens.extend(temp)
            tokens.append('</s>')
    else:
        temp=word_tokenize(sent_tokens)
        tokens.append('<s>')
        tokens.extend(temp)
        tokens.append('</s>')
    return tokens

In [103]:
vocab=[]
train_tokens=gettoken(text_train)
vocab=np.unique(train_tokens)
output=[]
for i in range(len(text_test)):
    output.append(text_test[i])
# vocab=np.unique(np.concatenate((vocab,np.array(token1)), axis=None))
# print(len(vocab))

In [112]:
def getgram(n):
    grams = list(everygrams(train_tokens,max_len=n))
    mle = MLE(n)
    mle.fit([grams], vocabulary_text=vocab)
    return mle
for i in range(1,5):
    print("No of ",i,"-gram Theoretically possible: ",len(vocab)**i)
    print("No of ",i,"-gram Practically: ",len(set(ngrams(train_tokens,i))))
unigram=getgram(1)
bigram=getgram(2)
trigram=getgram(3)
quadgram=getgram(4)
# print("lo")

No of  1 -gram Theoretically possible:  4277
No of  1 -gram Practically:  4277
No of  2 -gram Theoretically possible:  18292729
No of  2 -gram Practically:  28467
No of  3 -gram Theoretically possible:  78238001933
No of  3 -gram Practically:  53213
No of  4 -gram Theoretically possible:  334623934267441
No of  4 -gram Practically:  74296


In [108]:
def generate(n,model):
    start="<s>"
    gen_sent=[start]
    if(n==1):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==2):
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
    elif(n==3):
        start_arr=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        start_arr.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,start_arr) for i in vocab]))
            gen_sent.append(vocab[index])
            start=vocab[index]
            start_arr=start_arr[1:]
    elif(n==4):
        start_arr=[start]
        index=np.argmax(multi(1,[model.score(i,[start]) for i in vocab]))
        gen_sent.append(vocab[index])
        start_arr.append(vocab[index])
        index=np.argmax(multi(1,[model.score(i,start_arr) for i in vocab]))
        gen_sent.append(vocab[index])
        start_arr.append(vocab[index])
        while(start!="</s>"):
            index=np.argmax(multi(1,[model.score(i,start_arr) for i in vocab]))
            sent.append(vocab[index])
            start=vocab[index]
            start_arr=start_arr[1:]
    return ' '.join(sent)
print("Unigram:")
for i in range(10):
    print(generate(1,uni))
print("\nBigram:")
for i in range(10):
    print(generate(2,bi))
print("\nTrigram:")
for i in range(10):
    print(generate(3,tri))
print("\nQuadgram:")
for i in range(10):
    print(generate(4,quad))

Unigram:
<s> it ve <s> <s> anything can facilitator </s>
<s> a <s> and and <s> but the <s> </s>
<s> but were never think they do threat re saw don it reuters <s> we we days <s> you taken thing happens so you for companies are <s> building economic some very trump as going m to shutdown up them defend be i maybe person things have </s>
<s> </s>
<s> company on </s>
<s> some were </s>
<s> to line so of </s>
<s> comes happens cash i you actually t </s>
<s> to of </s>
<s> almost because mean <s> tell in <s> want <s> remain because <s> are exactly they go re sending </s>

Bigram:
<s> you re not look at the country </s>
<s> and just great answer i m running </s>
<s> they re losing this </s>
<s> believe it rich in jail </s>
<s> and i want them to build a real checks like this </s>
<s> a shame tell you </s>
<s> ill be someplace else </s>
<s> so i think he is when they re going to cruz </s>
<s> people stand back </s>
<s> this country rich </s>

Trigram:
<s> this guy </s>
<s> so we i </s>
<s> bec

In [109]:
t=[randrange(0,len(output_sent),1) for j in range(10)]
def perplex(models):
    for i in t:
        print("PP of ",' '.join(output[i]))
        for j in range(len(models)):
            mle=models[j]
            try:
                print(j+1," Gram ",round(lm.perplexity(ngrams(output_sent[i],j+1)),2))
            except:
                print(j+1," Gram ","inf")
perplex([unigram,bigram,trigram,quadgram])

NameError: name 'output_sent' is not defined